In [ ]:
import pandas as pd
import requests
import time


def get_geocode_from_publisher(University):
    # Replace with your actual Google Maps Geocoding API key
    GOOGLE_MAPS_API_KEY = "Insert your Google Maps Geocoding API key here"
    GEOCODE_API_URL = "https://maps.googleapis.com/maps/api/geocode/json"

    parameters = {
        "address": University,
        "key": GOOGLE_MAPS_API_KEY
    }

    try:
        response = requests.get(GEOCODE_API_URL, params=parameters)
        response.raise_for_status()
    except requests.exceptions.RequestException as err:
        print ("Oops: Something went wrong",err)
        return None, None

    data = response.json()

    if data["status"] == "OK":
        location = data["results"][0]["geometry"]["location"]
        return location["lat"], location["lng"]
    else:
        return None, None

# Read the CSV
df = pd.read_csv('analysed_data/Aff_individual_output_with_true_mean_as_base.csv', sep=';')

# Initialize empty lists for lat and lon
lat_list = []
lon_list = []

# Loop through each row
for aff_num in df['aff_num']:
    lat, lon = get_geocode_from_publisher(aff_num)
    lat_list.append(lat)
    lon_list.append(lon)
    time.sleep(1)  # sleep for 1 second to avoid rate limit

# Add latitude and longitude to dataframe
df['latitude'] = lat_list
df['longitude'] = lon_list

# Write the dataframe to a new CSV file
df.to_csv('analysed_data/affiliation_with_geocode.csv', index=False, sep=';')
